# Manual Mesh Generation

1.  Unit square in two dimensions

In [12]:
from netgen.meshing import *
ngmesh = Mesh(dim=2)

In [4]:
from netgen.meshing import *
ngmesh = Mesh(dim=2)
# Add MeshPoint 
N=5
pnums = []
for i in range(N + 1):
    for j in range(N + 1):
        pnums.append(ngmesh.Add(MeshPoint(Pnt(i / N, j / N, 0))))

# Create a region
idx_dom = ngmesh.AddRegion("mat", dim=2)
# Crate Elements
for j in range(N):
    for i in range(N):
        ngmesh.Add(Element2D(idx_dom, [pnums[i + j * (N + 1)],
                               pnums[i + (j + 1) * (N + 1)],
                               pnums[i + 1 + (j + 1) * (N + 1)],
                               pnums[i + 1 + j * (N + 1)]]))

# Boundary Elements
# horizontal boundaries
for i in range(N):
   ngmesh.Add(Element1D([pnums[N + i * (N + 1)],
                       pnums[N + (i + 1) * (N + 1)]], index=1))
   ngmesh.Add(Element1D([pnums[0 + i * (N + 1)], pnums[0 + (i + 1) * (N + 1)]], index=1))
ngmesh.SetBCName(0,"horizontal")
# help(Element1D)
# vertical boundaries
for i in range(N):
   ngmesh.Add(Element1D([pnums[i], pnums[i + 1]], index=2))
   ngmesh.Add(Element1D([pnums[i + N * (N + 1)], pnums[i + 1 + N * (N + 1)]], index=2))
ngmesh.SetBCName(1,"vertical")

In [13]:
import ngsolve
from ngsolve.webgui import Draw
mesh = ngsolve.Mesh(ngmesh)
Draw(mesh)

print(mesh.GetBoundaries())

/usr/lib/python3/dist-packages/jupyter_client/session.py:716: UserWarning: Message serialization failed with:
Out of range float values are not JSON compliant
Supporting this message is deprecated in jupyter-client 7, please make sure your message is JSON-compliant
  content = self.pack(content)


WebGuiWidget(layout=Layout(height='500px', width='100%'), value={'gui_settings': {}, 'ngsolve_version': '6.2.2…

()


In [14]:
from netgen.meshing import *
from ngsolve import Mesh as NGSMesh
import math

ngmesh = Mesh(dim=2)
r = 1.0

# 1. Puntos: Centro y Vértices
pcentral = ngmesh.Add(MeshPoint(Pnt(0, 0, 0)))

p_vertices = []
for i in range(5):
    # Ángulos para posicionar los 5 vértices
    theta = (2 * math.pi * i / 5) + (math.pi / 2)
    px, py = r * math.cos(theta), r * math.sin(theta)
    p_vertices.append(ngmesh.Add(MeshPoint(Pnt(px, py, 0))))

# 2. Crear Región del Dominio
idx_dom = ngmesh.AddRegion("mat_pentagono", dim=2)

# 3. Crear Elementos Triangulares (2D)
# Conectamos el centro con cada par de vértices consecutivos
for i in range(5):
    v1 = p_vertices[i]
    v2 = p_vertices[(i + 1) % 5]
    print(type(v1))
    # En Element2D para triángulos se pasan solo 3 nodos
    ngmesh.Add(Element2D(idx_dom, [pcentral, v1, v2]))

# 4. Definir las 5 Aristas de la Frontera (1D) con nombres distintos
for i in range(5):
    # Asignamos un índice único del 1 al 5
    ngmesh.Add(Element1D([p_vertices[i], p_vertices[(i + 1) % 5]], index=i+1))

# 5. Nombrar las fronteras
nombres = ["lado_1", "lado_2", "lado_3", "lado_4", "lado_5"]
for i, nombre in enumerate(nombres):
    ngmesh.SetBCName(i, nombre)

# 6. Finalizar y convertir a NGSolve
mesh = NGSMesh(ngmesh)

# Comprobación de la estructura
print(f"Tipo de elementos: Triángulos")
print(f"Nombres de frontera: {mesh.GetBoundaries()}")

<class 'netgen.libngpy._meshing.PointId'>
<class 'netgen.libngpy._meshing.PointId'>
<class 'netgen.libngpy._meshing.PointId'>
<class 'netgen.libngpy._meshing.PointId'>
<class 'netgen.libngpy._meshing.PointId'>
Tipo de elementos: Triángulos
Nombres de frontera: ('lado_1', 'lado_2', 'lado_3', 'lado_4', 'lado_5')


In [66]:
def tri2d_to_netgen(Coordinates, Elements3, list_element_boundaries, boundary_name=None):
    if boundary_name is None:
        boundary_name = [f"boundary_{i+1}" for i in range(len(list_element_boundaries))]
    from netgen.meshing import Element0D, Element1D, Element2D, MeshPoint, FaceDescriptor, Mesh
    from netgen.csg import Pnt

    ngmesh = Mesh(dim=2)
    pnums = []

    # Añadir puntos
    for coord in Coordinates:
        pnums.append(ngmesh.Add(MeshPoint(Pnt(coord[0], coord[1], 0))))

    # Crear región
    idx_dom = ngmesh.AddRegion("mat", dim=2)

    # Añadir elementos 2D
    for elem in Elements3:
        ngmesh.Add(Element2D(idx_dom, [pnums[i] for i in elem]))

    # Añadir elementos de frontera 1D
    

    for k, Elements_boundary in enumerate(list_element_boundaries):
        ngmesh.Add (FaceDescriptor(surfnr=1,domin=1,bc=k+1))
        for i, elem in enumerate(Elements_boundary):
            ngmesh.Add(Element1D([pnums[elem[0]], pnums[elem[1]]], index=k+1))
        #name boundaries
        ngmesh.SetBCName(k,boundary_name[k])
    
    return ngmesh


In [129]:
import numpy as np
Coordinates = np.array([[0,0],[1.0,0],[1.0,1.0],[0.5,1.5],[0,1.0]])
Elements3 = np.array([[0,1,4],[1,2,4],[2,3,4]]) 
Elements_boundary = np.array([[0,1],[1,2],[2,3],[3,4],[4,0],[0,2],[2,4]])
values_boundaries = np.array([1,2,2,2,2,0,0])
Elebnd = Elements_boundary[np.where(values_boundaries>0)[0],:]
print(Elebnd)
bnd1= Elements_boundary[np.where(values_boundaries==1)[0],:]
bnd2= Elements_boundary[np.where(values_boundaries==2)[0],:]
boundaries = []
boundaries.append(bnd1)
boundaries.append(bnd2)
test_ngmesh = tri2d_to_netgen(Coordinates, Elements3, boundaries, boundary_name=["bc1","bc2"])
test_mesh = NGSMesh(test_ngmesh)
print(test_mesh.GetBoundaries())

Draw(test_mesh)

[[0 1]
 [1 2]
 [2 3]
 [3 4]
 [4 0]]
('bc1', 'bc2')


2. U mesh

In [74]:
import meshio
import numpy as np
import h5py
from netgen.meshing import *
from ngsolve import Mesh as NGSMesh

In [75]:
f = h5py.File('umesh_h3.h5', 'r')

# print(list(f.keys()))
coordinates_mesh = f['mesh/coordinates'][:]
elements_mesh    = f['mesh/topology'][:]

coordinates_boundaries = f['/boundaries/coordinates'][:]
elements_boundaries    = f['/boundaries/topology'][:]
values_boundaries      = f['/boundaries/values'][:]
print(np.unique(values_boundaries))

boundary_1 = np.where(f['/boundaries/values'][:] == 1)[0]
boundary_elements_1 = elements_boundaries[boundary_1]

boundary_2 = np.where(f['/boundaries/values'][:] == 2)[0]
boundary_elements_2 = elements_boundaries[boundary_2]

boundary_3 = np.where(f['/boundaries/values'][:] == 3)[0]
boundary_elements_3 = elements_boundaries[boundary_3]
# boundaries
boundaries = []; boundary_names = []
# wall
boundaries.append(boundary_elements_1)
boundary_names.append("wall")
# inflow
boundaries.append(boundary_elements_2)
boundary_names.append("inflow")
# outflow
boundaries.append(boundary_elements_3)
boundary_names.append("outflow")



FileNotFoundError: [Errno 2] Unable to synchronously open file (unable to open file: name = 'umesh_h3.h5', errno = 2, error message = 'No such file or directory', flags = 0, o_flags = 0)

In [17]:
U_ngmesh = tri2d_to_netgen(coordinates_mesh, elements_mesh, boundaries, boundary_names)
U_mesh = NGSMesh(U_ngmesh)
print(U_mesh.GetBoundaries())

('wall', 'inflow', 'outflow')


In [19]:
U_mesh.ngmesh.Save("U_mesh_refinada.vol")

3. Arteria. Read and Create in NGSolve

In [76]:
import meshio
import numpy as np
import h5py
from netgen.meshing import *
from ngsolve import Mesh as NGSMesh

In [77]:
f = h5py.File('../Test_mallas/mallas/Carotid_h1.h5', 'r')
# print(list(f.keys()))
coordinates_mesh = f['mesh/coordinates'][:]
elements_mesh    = f['mesh/topology'][:]

coordinates_boundaries = f['/boundaries/coordinates'][:]
elements_boundaries    = f['/boundaries/topology'][:]
values_boundaries      = f['/boundaries/values'][:]
# print(np.unique(values_boundaries))

In [78]:
boundary_1 = np.where(f['/boundaries/values'][:] == 1)[0]
boundary_elements_1 = elements_boundaries[boundary_1]

boundary_2 = np.where(f['/boundaries/values'][:] == 2)[0]
boundary_elements_2 = elements_boundaries[boundary_2]

boundary_3 = np.where(f['/boundaries/values'][:] == 3)[0]
boundary_elements_3 = elements_boundaries[boundary_3]

boundary_4 = np.where(f['/boundaries/values'][:] == 4)[0]
boundary_elements_4 = elements_boundaries[boundary_4]

boundary_5 = np.where(f['/boundaries/values'][:] == 5)[0]
boundary_elements_5 = elements_boundaries[boundary_5]

boundary_6 = np.where(f['/boundaries/values'][:] == 6)[0]
boundary_elements_6 = elements_boundaries[boundary_6]

boundary_7 = np.where(f['/boundaries/values'][:] == 7)[0]
boundary_elements_7 = elements_boundaries[boundary_7]

boundaries = [boundary_elements_1, boundary_elements_2,
             boundary_elements_3, boundary_elements_4,
              boundary_elements_5, boundary_elements_6,
              boundary_elements_7]

boundary_wall = boundary_1
boundary_inflow = boundary_2
boundary_outflow = np.concatenate([boundary_3, boundary_4, boundary_5, boundary_6, boundary_7])

# boundaries
boundaries = []; boundary_names = []
# wall
boundaries.append(boundary_elements_1)
boundary_names.append("wall")
# inflow
boundaries.append(boundary_elements_2)
boundary_names.append("inflow")
# outflow
boundaries.append(np.concatenate([boundary_elements_3, boundary_elements_4,
                                 boundary_elements_5, boundary_elements_6,
                                 boundary_elements_7]))
boundary_names.append("outflow")


In [ ]:
arteria_ngmesh = tri2d_to_netgen(coordinates_mesh, elements_mesh, boundaries, boundary_names)
arteria_mesh = NGSMesh(arteria_ngmesh)
print(arteria_mesh.GetBoundaries())

[array([[   0,  652],
       [   0, 3173],
       [   1,   42],
       ...,
       [3328, 3330],
       [3328, 3331],
       [3331, 3332]]), array([[ 592, 1296],
       [ 592, 2966],
       [1296, 1297],
       [1866, 2074],
       [1866, 2087],
       [1880, 1881],
       [1880, 2839],
       [1881, 1882],
       [1882, 2074],
       [2087, 2088],
       [2088, 2966],
       [2838, 2839]]), array([[  28,   29],
       [  29,   72],
       [ 152,  154],
       [ 154,  174],
       [ 108,  109],
       [ 108,  181],
       [ 109,  110],
       [ 110, 2732],
       [2626, 2732],
       [ 748,  749],
       [ 748, 3157],
       [ 749,  750],
       [ 750, 2841],
       [2841, 2842],
       [2603, 2604],
       [2604, 2605]])]
['wall', 'inflow', 'outflow']
('wall', 'inflow', 'outflow')


In [142]:
Draw(arteria_mesh)

WebGuiWidget(layout=Layout(height='500px', width='100%'), value={'gui_settings': {}, 'ngsolve_version': '6.2.2…

BaseWebGuiScene

In [144]:
arteria_mesh.ngmesh.Save("arteria_mesh.vol")

# U mesh format .obj

In [80]:
import meshio
import numpy as np
from netgen.meshing import *
from ngsolve import Mesh as NGSMesh

def get_mesh_data(file_path):
    # Load mesh
    malla = meshio.read(file_path)
    Coordinates = malla.points
    Elements = malla.cells_dict["triangle"]
    
    # Generate all edges
    all_edges = np.sort(np.concatenate([
        Elements[:, [0, 1]],
        Elements[:, [1, 2]],
        Elements[:, [2, 0]]
    ]), axis=1)
    
    # Find unique and boundary edges
    Edges, counts = np.unique(all_edges, axis=0, return_counts=True)
    BoundaryEdges = Edges[counts == 1]
    
    return Coordinates, Elements, Edges, BoundaryEdges

Coordinates, Elements, Edges, BoundaryEdges = get_mesh_data("Ugeom.obj")

In [84]:
pared   = []; salida  = []; entrada = []
for i,facets_e in enumerate(BoundaryEdges):
    nodox = Coordinates[facets_e,0]
    nodoy = Coordinates[facets_e,1]
    m = np.array([np.mean(nodox), np.mean(nodoy)])
    if m[1] > -2.0:
        pared.append(facets_e)
    if m[1] <= -2.0 and m[0] > 0.0:
        salida.append(facets_e)
    if m[1] <= -2.0 and m[0] < 0.0:
        entrada.append(facets_e)
Boundary_wall = np.array(pared)
Boundary_outflow = np.array(salida)
Boundary_inflow = np.array(entrada)

Boundaries = []
Boundaries.append(Boundary_wall)
Boundaries.append(Boundary_inflow)
Boundaries.append(Boundary_outflow)

boundary_names = ["wall", "inflow", "outflow"]

In [89]:
U_ngmesh = tri2d_to_netgen(Coordinates, Elements, Boundaries, boundary_names)
U_mesh = NGSMesh(U_ngmesh)
U_mesh.ngmesh.Save("U_mesh.vol")

In [88]:
from ngsolve.webgui import Draw
Draw(U_mesh)

WebGuiWidget(layout=Layout(height='500px', width='100%'), value={'gui_settings': {}, 'ngsolve_version': '6.2.2…

BaseWebGuiScene